<a href="https://colab.research.google.com/github/vishnu-chand/starDetection/blob/main/starDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

import os
if '' or not os.path.exists('/content/pixUtils'):
    !git clone https://github.com/vishnu-chand/pixUtils.git;mv pixUtils deleteMe;mv deleteMe/pixUtils .;rm -rf deleteMe
    from pixUtils import *
    downloadDB('wget+https://unblast.com/wp-content/uploads/2018/10/Sky-Stars-Pattern-1600x1190.jpg', '/content/db')
    downloadDB('wget+https://images-assets.nasa.gov/image/iss006e28068/iss006e28068~medium.jpg', '/content/db')

from pixUtils import *
from ipywidgets import interact 
from IPython.display import display
from PIL.Image import fromarray as Img

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def generateStar():
    s = np.random.randint(720, 1280)
    x = np.random.random_sample([s, s, 3])
    x[x < .999] = 0
    x[x != 0] = 1
    x = x.astype('u1')
    return x * 255


def getHarrisCorners(x, k, kernel_size):
    """
    s1: implement sliding window
    s2: on every window differentiate image in x-axis and y-axis (dx, dy)
    s3: stack dx and dy and get covariance matirx S # structural tensor
    s4: calcualte harris response which is det(S) - k * sq(trace(S)); where k is precision factor
    s5: take top N% of corrects which will stars
    algorithm can be improved by adding non maxima supression
    """
    # k -> precision
    # kernel_size -> aperture
    res = np.zeros_like(x).astype('f4')
    center = kernel_size // 2
    dy, dx = cv2.Sobel(x, cv2.CV_64F, 0, 1, ksize=3), cv2.Sobel(x, cv2.CV_64F, 1, 0, ksize=3)
    nRow, nCol = x.shape
    for r in np.arange(nRow - kernel_size + 1):
        for c in np.arange(nCol - kernel_size + 1):
            sdy, sdx = dy[r:r + kernel_size, c:c + kernel_size], dx[r:r + kernel_size, c:c + kernel_size]
            a = np.stack([sdy.ravel(), sdx.ravel()], axis=1)
            (sxx, sxy), (syx, syy) = a.T.dot(a)  # gradient covariance
            det = sxx * syy - sxy * syx
            trace = sxx + syy
            harrisResponse = det - k * (trace ** 2)
            res[r + center, c + center] = harrisResponse
    return res

In [ ]:
# oyImg = cv2.imread('/content/db/Sky-Stars-Pattern-1600x1190.jpg')
oyImg = cv2.imread('/content/db/iss006e28068~medium.jpg')
# oyImg = generateStar()
oyImg = cv2.copyMakeBorder(oyImg, 5, 5, 5, 5, cv2.BORDER_CONSTANT, None, 0)
x = cv2.cvtColor(oyImg, cv2.COLOR_BGR2GRAY)
oyHat = getHarrisCorners(x, 0.06, 2)
# oyHat = cv2.cornerHarris(x, 2, 3, 0.04)

In [ ]:
def detectStar(threshold):
    yImg, yHat = oyImg.copy(), oyHat.copy()
    yHat = np.uint8(yHat > threshold * yHat.max())  # consider corners vales bigger than 0.0001 peak
    for x0, y0 in zip(*np.where(yHat > 0)):
        cv2.circle(yImg, (y0, x0), 10, (0, 255, 0), thickness=1)
    display(Img(yImg))

interact(detectStar, threshold=(0.001, 1.0, .1))

interactive(children=(FloatSlider(value=0.401, description='threshold', max=1.0, min=0.001), Output()), _dom_c…

<function __main__.detectStar>